Urban Data Science & Smart Cities <br>
URSP688Y Spring 2025<br>
Instructor: Chester Harvey <br>
Urban Studies & Planning <br>
National Center for Smart Growth <br>
University of Maryland

# Exercise03

## Problem

In the last exercise, you explored 



## Data



## File Management and Submitting


## Getting Started

https://geocoding.geo.census.gov/geocoder/

In [1]:
import pandas as pd
import geopandas as gpd
import utils
import exercise03
import census_geocode

%load_ext autoreload
%autoreload 2

In [2]:
# Load warrents and make sure zip codes are stored as strings without decimals
warrents_df = pd.read_csv('md_eviction_warrents_through_2024.csv')
warrents_df['TenantZipCode'] = warrents_df['TenantZipCode'].astype('Int64').astype('string')
len(warrents_df)

411040

In [3]:
geocode_input_df = exercise03.prep_warrents_for_geocoding(warrents_df)

411040 warrents input
Reduced to 167949 unique addresses


In [4]:
# Split into dataframes with less than 10000 rows each
geocode_input_dfs = utils.chunk_dataframe(geocode_input_df, 9999)

# Save each dataframe as a CSV without a header
utils.save_dfs_to_csv(geocode_input_dfs, 'geocode_inputs', header=False)

split dataframe into 17 chunks


In [5]:
# Geocode addresses with the Census Geocoder
# census_geocode.geocode_csvs('geocode_inputs', 'geocode_outputs', test=True)

In [6]:
# Recombine outputs from geocoder into a single dataframe
geocode_output_df = exercise03.combine_census_geocoded_csvs('geocode_outputs')
len(geocode_output_df)

167949

In [7]:
# Merge geocoded address back onto the inputs with separate fields for address, city, state, and zip
geocoded_df = geocode_input_df.merge(geocode_output_df, left_index=True, right_index=True)
len(geocoded_df)

167949

In [8]:
# Use address, city, state, and zip columns to join geocodes onto original warrent records
warrents_df = warrents_df.merge(geocoded_df, on=['TenantAddress','TenantCity','TenantState','TenantZipCode'])
len(warrents_df)

411040

In [9]:
# Convert warrents into a geodataframe with points
warrents_gdf = utils.lonlat_str_to_geodataframe(warrents_df, 'match_lon_lat')

In [11]:
# What proportion of records have points?
len(warrents_gdf[warrents_gdf.lon.notnull()]) / len(warrents_gdf)

0.9458057609964967

In [12]:
# What proportion of records have exact geocode matches?
len(warrents_gdf[warrents_gdf.match_type == 'Exact']) / len(warrents_gdf)

0.5598554885169327